# Glowpick top100

In [61]:
import pandas as pd
import re

In [66]:
df = pd.read_excel('./cj_oliveyoung_shampoo_edit.xlsx')
len(set(df['name']))

103

In [67]:
set(df['name'])

{'TS 프리미엄 샴푸 300ml2+100ml 기획',
 '[온라인단독] 달리프 클로렐라 베러 루트 릴렉싱샴푸 500ml*2개 (1+1기획세트)',
 '[온라인단독] 라우쉬 씨위드 디그리징 샴푸 200ML & 씨위드 스칼프 팩 100ML 기획',
 '[온라인단독] 아토팜 마일드 샴푸 300ml',
 '[온라인단독] 클로란 망고 샴푸 400ml 2+1',
 '[온라인단독] 클로란 퀴닌 샴푸 400ml 2+1',
 '[온라인단독] 클로란 퀴닌 샴푸 400ml*2 +헤어 브러쉬 기획',
 '[온라인단독]더마클라센 굿바이 헤어 로스 샴푸 레몬그라스향 1000ml 기획(탈모완화샴푸)',
 '[온라인단독]더마클라센 굿바이 헤어 로스 샴푸 로즈마리향 1000ml 기획 (탈모완화샴푸)',
 '[온라인단독]더프트앤도프트 소피소피 퍼퓸드 헤어 샴푸 500ml',
 '[온라인단독]더프트앤도프트 핑크브리즈 퍼퓸드 헤어 샴푸 500ml',
 '[온라인단독]록시땅 아로마 리페어 샴푸 300ml',
 '[온라인단독]록시땅 아로마 리페어 샴푸 500ml',
 '[온라인단독]미쟝센 데미지케어샴푸 680ml',
 '[온라인단독]미쟝센 퍼펙트 리페어 샴푸 680ML',
 '[온라인단독]이브로쉐 호호바 리페어 샴푸 300ML 2+1 기획_',
 '[온라인단독]착한팩토리 꿀폭탄 샴푸1000ml+트리트먼트1000ml',
 '[온라인단독]클로란 피오니 샴푸 400ml 2+1',
 '[온라인전용]클로란 일랑일랑 무실리콘 너리싱 샴푸 200ml',
 '[클로란] 오트밀크 울트라 젠틀 샴푸 400ml',
 '달리프 클로렐라 베러 루트 릴렉싱샴푸 500ml',
 '더마앤모어 미셀라 안티더스트 두피 샴푸 600ml',
 '더마클라센 굿바이헤어로스 샴푸 레몬그라스 기획세트l+100ml+트리트먼트 35ml]',
 '더마클라센 샴푸 1,000ml 레몬글라스 + 트리트먼트 200ml 기획',
 '더프트앤도프트 소피소피 샴푸 트리트먼트 기획 세트',
 '라우쉬 스위스 허벌 케어 샴푸 200ml(수분 진정 샴푸)',
 

In [2]:
top100 = pd.read_excel('shampoo_top100_list.xlsx')
top100.head()

,Unnamed: 0,brand,name,url,joiner
0,0,아모스프로페셔널,녹차실감 샴푸액 지성용,https://www.glowpick.com/product/16977,873
1,1,존프리다(JOHN FRIEDA),프리즈 이즈 미라큘러스 리커버리 샴푸,https://www.glowpick.com/product/82542,85
2,2,아모스프로페셔널,녹차실감 샴푸액 중건성용,https://www.glowpick.com/product/16978,370
3,3,하이앙포레,프리미엄 샴푸,https://www.glowpick.com/product/20347,178
4,4,암웨이,새티니크 글로시 리페어 샴푸,https://www.glowpick.com/product/22448,99


In [3]:
len(set(top100['brand']))

60

In [4]:
top60 = list(set(top100['brand']))
pattern = re.compile('[^가-힣+]') 
top60_new = [pattern.sub('', brand) for brand in top60]
top60_new = ['TS' if x=='' else x for x in top60_new]

# Review Scraping

In [127]:
import selenium
from selenium import webdriver
import time
from bs4 import BeautifulSoup
import re
from selenium.webdriver.common.keys import Keys

In [128]:
url = 'http://display.cjmall.com/p/specialtyShop/S01552?subGnbId=CATEGORY'

In [130]:
# open the webbrowser
driver = webdriver.Chrome()
driver.get(url)
time.sleep(2)

links_total = []
for brand in top60_new:
    try:
        # enter the keyword in the searchbox
        search_box = driver.find_element_by_id('re_srch0')
        search_box.send_keys(brand)
        search_click = driver.find_element_by_id('btn_re_srch0').click()
        time.sleep(2)

        # select item
        link_elements = driver.find_elements_by_css_selector('ul.lst_cate_result li > a')
        links = [i.get_attribute('href') for i in link_elements]
        links_total.extend(links)
        driver.back()
        search_box.clear()
        
    except Exception as e:
        print(e)
        break        

In [204]:
links_total = list(set(links_total))
len(links_total)

171

In [205]:
df_sum = pd.DataFrame()

driver = webdriver.Chrome()
for link in links_total:
#     driver = webdriver.Chrome()
    driver.get(link)
    time.sleep(2)

    ## get goods' name, price, stars, reviews
    # name
    name = driver.find_element_by_css_selector('h3.prd_tit').text.strip()

    # price
    price = driver.find_element_by_css_selector('span.price_txt > strong.ff_price').text

    # rating & review
    premium_rating = []
    premium_review = []
    regular_rating = []
    regular_review = []

    # preminum review
    while True:

        try: 
            ratings = driver.find_elements_by_css_selector('div.review_premium ul.review_lst li span.blind')
            rating_ls = [rating.get_attribute('innerHTML') for rating in ratings]
            premium_rating.extend(rating_ls)

            reviews = driver.find_elements_by_class_name('review_scr._brTag')
            review_ls = [review.get_attribute('innerHTML') for review in reviews]
            premium_review.extend(review_ls)

            driver.find_element_by_css_selector('div.review_premium a.btn_pn_next').click()
            time.sleep(2)

        except Exception as e:
            print(e)
            break

    # review
    while True:

        try: 
            ratings = driver.find_elements_by_css_selector('div.review_line ul.review_lst li span.blind')
            rating_ls = [rating.get_attribute('innerHTML') for rating in ratings]
            regular_rating.extend(rating_ls)

            reviews = driver.find_elements_by_css_selector('div.review_line ul.review_lst li p.review_scr')
            review_ls = [review.get_attribute('innerHTML') for review in reviews]
            regular_review.extend(review_ls)

            driver.find_element_by_css_selector('div.review_line a.btn_pn_next').click()
            time.sleep(2)

        except Exception as e:
            print(e)
            break
    d = {'name':name, 'price':price, 'rating':premium_rating + regular_rating, 'revivew':premium_review + regular_review,}
    df = pd.DataFrame(data=d)
    df_sum = df_sum.append(df)
    
    # back to previous page
    driver.back()
    time.sleep(3)

Message: no such element: Unable to locate element: {"method":"css selector","selector":"div.review_premium a.btn_pn_next"}
  (Session info: chrome=77.0.3865.90)

Message: no such element: Unable to locate element: {"method":"css selector","selector":"div.review_line a.btn_pn_next"}
  (Session info: chrome=77.0.3865.90)

Message: no such element: Unable to locate element: {"method":"css selector","selector":"div.review_premium a.btn_pn_next"}
  (Session info: chrome=77.0.3865.90)

Message: no such element: Unable to locate element: {"method":"css selector","selector":"div.review_line a.btn_pn_next"}
  (Session info: chrome=77.0.3865.90)

Message: no such element: Unable to locate element: {"method":"css selector","selector":"div.review_premium a.btn_pn_next"}
  (Session info: chrome=77.0.3865.90)

Message: no such element: Unable to locate element: {"method":"css selector","selector":"div.review_line a.btn_pn_next"}
  (Session info: chrome=77.0.3865.90)

Message: no such element: Unabl

In [208]:
df_sum

,name,price,rating,revivew
0,츠바키 볼륨터치 샴푸 500ml,"15,000",100점,츠바키 친구 추천으로 한번 구입해 봤다가 정착템 인생샴푸 되었습니다 ㅋㅋㅋ 드라이로...
1,츠바키 볼륨터치 샴푸 500ml,"15,000",100점,무실리콘 샴푸인데도 샴푸후 뻣뻣함도 없고 부드럽고 향기도 좋고 마음에 듭니다. 보라...
2,츠바키 볼륨터치 샴푸 500ml,"15,000",100점,"원래 빨간색 용기 츠바키를 사용하다가 품절이라 그냥 질렀어요! 배송은 빨랐고요, ..."
3,츠바키 볼륨터치 샴푸 500ml,"15,000",100점,"한번 사용해보고 만족스러워서 재구매합니다. 향도 좋고, 무엇보다 머리가 부들부들 해..."
4,츠바키 볼륨터치 샴푸 500ml,"15,000",100점,점점 가늘어지는 모발에 처음으로 구입해 사용해보았어요. 향도 좋구 거품도 잘 나서 ...
5,츠바키 볼륨터치 샴푸 500ml,"15,000",85점,세일기간에 산 제품이라 배송 오래걸린거 말고는 만족스럽습니다\n<br>샴푸린스 세트...
6,츠바키 볼륨터치 샴푸 500ml,"15,000",100점,"2-3달마다 펌과 한달에 한번 흰머리 염색을 하는지라 극손상모인데, 머리빠짐도 심하..."
7,츠바키 볼륨터치 샴푸 500ml,"15,000",95점,헤어워터 잘 사용하고 있어서 샴푸도 구매해봤는데 그냥그냥이에요. 머리결은 좋아지는것...
8,츠바키 볼륨터치 샴푸 500ml,"15,000",100점,숱이없어서\n<br>볼륨상푸자주쓰는데\n<br>츠바키는 하얀색만\n<br>쓰다가\n...
9,츠바키 볼륨터치 샴푸 500ml,"15,000",100점,빠른배송감사드리고요 샴푸도 너무 만족스럽스니다 와이프가 사용해보고 무척 만적스러워 ...


# File export

In [211]:
df_sum.to_excel('./cj_oliveyoung.xlsx')

In [24]:
df_sum = pd.read_excel('cj_oliveyoung.xlsx', index_col=False)
df_sum.rename(columns={'revivew':'review'})
df_sum = df_sum[['name','price','rating','revivew']]

In [39]:
df_sum = df_sum[df_sum['name'].str.contains('샴푸')]
df_sum = df_sum.reset_index(drop=True)

In [40]:
df_sum.to_excel('./cj_oliveyoung_shampoo.xlsx')

In [11]:
# scale rating to 1-5
df_sum = pd.read_excel('cj_oliveyoung_shampoo.xlsx', index_col=False)
df_sum = df_sum[['name','price','rating','revivew']]

In [44]:
new_rating=[]
rating = [int(re.sub('점','',x)) for x in df_sum['rating']]
for x in rating:
    if x >= 80: 
        new_rating.append(5)
    elif x >= 60:
        new_rating.append(4)
    elif x >= 40:
        new_rating.append(3)
    elif x >= 20:
        new_rating.append(2)
    elif x >= 0:
        new_rating.append(1)
    else: pass

In [55]:
df_sum['rating'] = new_rating

In [59]:
df_sum.to_excel('./cj_oliveyoung_shampoo_edit.xlsx')